In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests

import warnings
warnings.filterwarnings('ignore')

import os
import selenium
from selenium import webdriver
import re
from datetime import datetime as dt, timedelta
import datetime
from tqdm import tqdm

import pandas as pd
import time
import math
import numpy as np

from dateutil.parser import parse
from selenium.webdriver import Chrome, ChromeOptions
import lxml
import lxml.html

In [2]:
job_list = pd.read_excel('./잡코리아 직무페이지.xlsx')
job_list

,직무,공고수,페이지수,링크
0,경영 사무,1451,73,https://www.jobkorea.co.kr/starter/PassAssay?F...
1,마케팅 광고 홍보,228,12,https://www.jobkorea.co.kr/starter/PassAssay?F...
2,IT 인터넷,1007,51,https://www.jobkorea.co.kr/starter/PassAssay?F...
3,디자인,23,2,https://www.jobkorea.co.kr/starter/PassAssay?F...
4,무역 유통,515,26,https://www.jobkorea.co.kr/starter/PassAssay?F...
5,영업 고객상담,1455,73,https://www.jobkorea.co.kr/starter/PassAssay?F...
6,서비스,114,6,https://www.jobkorea.co.kr/starter/PassAssay?F...
7,연구개발 설계,988,50,https://www.jobkorea.co.kr/starter/PassAssay?F...
8,생산 제조,730,37,https://www.jobkorea.co.kr/starter/PassAssay?F...
9,교육,20,1,https://www.jobkorea.co.kr/starter/PassAssay?F...


In [ ]:
comp_page_list

In [ ]:
comp_page_list = pd.DataFrame(columns=['직무명', '회사명','공고링크'])

In [38]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
op.add_argument('user-agent=' + user_agent)
browser = Chrome('./chromedriver.exe', options=op)
browser.get('https://www.jobkorea.co.kr/')
time.sleep(15)

not_list = []
for i in tqdm(range(len(job_list))):
    job = job_list['직무'][i]
    for j in range(1, job_list['페이지수'][i]+1):
        url = job_list['링크'][i] + str(j)
        browser.get(url)
        time.sleep(1)
        html = browser.page_source
        root = lxml.html.fromstring(html)
        
        for k in range(1, 21):
            try:
                comp_page_root = root.cssselect(f'#container > div.stContainer > div.starListsWrap.ctTarget > ul > li:nth-child({k}) > div.txBx > p > a')[0].get('href')
                comp_page = 'www.jobkorea.co.kr' + comp_page_root
                comp_name= root.cssselect(f'#container > div.stContainer > div.starListsWrap.ctTarget > ul > li:nth-child({k}) > div.txBx > p > a > span')[0].text_content()
                
                data_to_insert = {'직무명': job, '회사명':comp_name, '공고링크':comp_page}
                comp_page_list = comp_page_list.append(data_to_insert, ignore_index=True)

            except:
                break


100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [24:30<00:00, 105.05s/it]


In [39]:
comp_page_list

,직무,회사명,공고링크
0,마케팅 광고 홍보,(주)푸르밀,www.jobkorea.co.kr/starter/PassAssay/View/2375...
1,마케팅 광고 홍보,신한카드(주),www.jobkorea.co.kr/starter/PassAssay/View/2376...
2,마케팅 광고 홍보,LG디스플레이,www.jobkorea.co.kr/starter/PassAssay/View/2375...
3,마케팅 광고 홍보,(주)현대백화점,www.jobkorea.co.kr/starter/PassAssay/View/2363...
4,마케팅 광고 홍보,(주)현대렌탈케어,www.jobkorea.co.kr/starter/PassAssay/View/2357...
...,...,...,...
7117,전문 특수직,골프존뉴딘홀딩스,www.jobkorea.co.kr/starter/PassAssay/View/1440...
7118,전문 특수직,(주)네오위즈,www.jobkorea.co.kr/starter/PassAssay/View/1441...
7119,전문 특수직,(주)셀트리온,www.jobkorea.co.kr/starter/PassAssay/View/1441...
7120,전문 특수직,(주)웹젠,www.jobkorea.co.kr/starter/PassAssay/View/1441...


In [59]:
comp_job_opening = pd.DataFrame(columns=['회사명','직무대분류','직무명','내용','스펙'])

In [65]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
op.add_argument('user-agent=' + user_agent)
browser = Chrome('./chromedriver.exe', options=op)
browser.get('https://www.jobkorea.co.kr/')
time.sleep(20)

not_list = []
for i in tqdm(range(len(comp_page_list))):
    try:
        big_job = comp_page_list['직무명'][i]
        comp_name = comp_page_list['회사명'][i]
        url = comp_page_list['공고링크'][i]
        browser.get('https://' + url)
        time.sleep(1)
        html = browser.page_source
        root = lxml.html.fromstring(html)

        job_root = root.cssselect('#container > div.stContainer > div.selfTopBx > div.viewTitWrap > h2 > em')[0].text_content()
        job = job_root.split()[-1]

        comp_answer = []
        for k in range(2,12,2):
            try:
                answer = root.cssselect(f'#container > div.stContainer > div.selfQnaWrap > dl > dd:nth-child({k}) > div')[0].text_content()
                comp_answer.append(answer)
            except:
                break

        final_answer = ' '.join(comp_answer)

        data_to_insert = {'회사명': comp_name, '직무대분류':big_job, '직무명':job, '내용':final_answer}
        comp_job_opening = comp_job_opening.append(data_to_insert, ignore_index=True)
        
    except:
        not_list.append(comp_name)
        print(comp_name)
        time.sleep(0.5)


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [11:39<00:15,  7.69s/it]

(주)인터파크


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [11:50<00:00,  7.11s/it]


In [66]:
comp_job_opening

,회사명,직무대분류,직무명,내용,스펙
0,(주)보쉬렉스로스코리아,무역 유통,유통·물류·재고,\nTo whom it may concern\n\nI was excited to s...,NaN
1,중소벤처기업진흥공단,무역 유통,해외영업·무역영업,"\n[초보 스페인어 통역가, 최선의 노력] \n영어 통역 담당으로 참가했던 국제 보...",NaN
2,대보건설(주),무역 유통,해외영업·무역영업,\n우리나라 서비스산업은 최근 내수활성화 뿐만 아니라 제조업의 경쟁력 향상을 위해서...,NaN
3,고려해운(주),무역 유통,해외영업·무역영업,\n[동반성장]\n세계규모 경기침체와 고유가 시대에 들어서며 휘청거리는 크고 작은 ...,NaN
4,넥스트엔터테인먼트월드,무역 유통,해외영업·무역영업,"\n- 살면서 쉬웠던 날은 단 하루도 없었다 (박광수, 예담) : \n삶의 희노애락...",NaN
...,...,...,...,...,...
794,케어캠프(주),영업 고객상담,영업관리·지원·영업기획,"\n""책임 있는 봉사""\n\n힘든 환경의 사람들을 도와주면 저의 가치를 느끼며 마음...",NaN
795,㈜우리은행,영업 고객상담,금융·보험영업,"\n[아침 5시의 기적, OOO의 기적]\n\n공부에 흥미를 갖지 못했던 중학교 시...",NaN
796,롯데칠성음료(주),영업 고객상담,제품·서비스영업,\n[문화를 이끄는 롯데의 영업인]\n\n먹고 마시는 식은 인간의 삶에서 꼭 필요한...,NaN
797,코웨이㈜,영업 고객상담,영업관리·지원·영업기획,"\n[설득을 통해 도전적인 시도를 할 수 있었고, 좋은 성적을 이끌어냈습니다]\n\...",NaN
